In [23]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Attention

# Load the IMDb movie review dataset


imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

# Preprocess the data
max_sequence_length = 200  # Maximum sequence length for padding
train_data = pad_sequences(train_data, maxlen=max_sequence_length)
test_data = pad_sequences(test_data, maxlen=max_sequence_length)

# Define the model architecture
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(10000, 128)(input_layer)
conv_layer = Conv1D(64, 5, activation='relu')(embedding_layer)
pooling_layer = MaxPooling1D(pool_size=4)(conv_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=False))(pooling_layer)
#lstm_layer = Bidirectional(LSTM(64, return_sequences=False), name="encoder_lstm")(pooling_layer)
#attention_layer = Attention()(lstm_layer)
attention_layer = Attention()([lstm_layer, lstm_layer])
output_layer = Dense(1, activation='sigmoid')(attention_layer)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_data, train_labels, batch_size=32, epochs=5, validation_data=(test_data, test_labels))

# Evaluate the model
loss, accuracy = model.evaluate(test_data, test_labels)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

# Predict on test data
predictions = model.predict(test_data)

# Convert probabilities to class labels
predicted_labels = np.round(predictions).flatten()

# Create confusion matrix
confusion_matrix = pd.crosstab(test_labels, predicted_labels, rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)


Epoch 1/5
782/782 [==============================] - 45s 54ms/step - loss: 0.6934 - accuracy: 0.4988 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/5
782/782 [==============================] - 41s 53ms/step - loss: 0.6159 - accuracy: 0.6060 - val_loss: 0.4002 - val_accuracy: 0.8270
Epoch 3/5
782/782 [==============================] - 41s 53ms/step - loss: 0.3048 - accuracy: 0.8761 - val_loss: 0.3027 - val_accuracy: 0.8731
Epoch 4/5
782/782 [==============================] - 41s 53ms/step - loss: 0.1996 - accuracy: 0.9256 - val_loss: 0.3011 - val_accuracy: 0.8780
Epoch 5/5
782/782 [==============================] - 10s 12ms/step - loss: 0.3428 - accuracy: 0.8706
Test Loss: 0.34278905391693115
Test Accuracy: 0.8705599904060364
782/782 [==============================] - 10s 12ms/step
Predicted    0.0    1.0
Actual                 
0          11417   1083
1           2153  10347


In [13]:
imd_train= pd.read_csv("twitter_training.csv")
print(imd_train.head())
imd_test=pd.read_csv("twitter_validation.csv")
print(imd_test.head())

   2401  Borderlands  Positive  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

  im getting on borderlands and i will murder you all ,  
0  I am coming to the borders and I will kill you...     
1  im getting on borderlands and i will kill you ...     
2  im coming on borderlands and i will murder you...     
3  im getting on borderlands 2 and i will murder ...     
4  im getting into borderlands and i can murder y...     
   3364   Facebook Irrelevant  \
0   352     Amazon    Neutral   
1  8312  Microsoft   Negative   
2  4371      CS-GO   Negative   
3  4433     Google    Neutral   
4  6273       FIFA   Negative   

  I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣  
0